In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import csv

# Function to read FEA results from CSV file
def read_fea_results(filename):
    positions = []
    fea_strains = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Skip header row
        for row in reader:
            positions.append(float(row[0]))  # First column: positions
            fea_strains.append([float(strain) for strain in row[1:]])  # Strain values for each load case
    return np.array(positions), np.array(fea_strains)

filename = "beam_strain_data.csv"  # The CSV file containing FEA results
positions, fea_strains = read_fea_results(filename)
random_seed = np.random.seed(50)
true_load_case = 7
duration = 60
ground_truth = fea_strains[:,true_load_case]


strain_gauge_data = []
for _ in range(duration):
    noisy_data = ground_truth #+ np.random.normal(0, ground_truth * .02, ground_truth.shape)
    strain_gauge_data.append(noisy_data)



sensor_indices = [2, 5]  # Sensors placed at element 3 and 5

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Assuming strain_data is a list of shape (N_samples, 2) and labels are (N_samples,)
# strain_data = [[strain_at_point_2, strain_at_point_5], ...]
# labels = [load_case_1, load_case_2, ...]

strain_data = np.array([...])  # Fill with your strain data
labels = np.array([...])  # Fill with your corresponding load cases

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(strain_data, labels, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


In [ ]:
class StrainNN(nn.Module):
    def __init__(self):
        super(StrainNN, self).__init__()
        self.fc1 = nn.Linear(2, 64)  # Two strain inputs
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 15)  # 15 output classes (load cases)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Output logits
        return x


In [ ]:
# Initialize the neural network
model = StrainNN()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')


In [ ]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy on test data: {accuracy * 100:.2f}%')


In [ ]:
# Example input (new strain values at points 2 and 5)
strain_input = torch.tensor([[strain_2, strain_5]], dtype=torch.float32)

# Get probabilities
model.eval()
with torch.no_grad():
    output = model(strain_input)
    probabilities = torch.softmax(output, dim=1)
    print(probabilities)
